<img align="right" width="400" src="https://www.fhnw.ch/de/++theme++web16theme/assets/media/img/fachhochschule-nordwestschweiz-fhnw-logo.svg" alt="FHNW Logo">


# Data Augmentation

by Fabian Märki

## Summary
The aim of this notebook is to show some options for data augmentation in nlp (in contrast to e.g. [image augmentation](https://github.com/aleju/imgaug)).

### Sources
- [Data Augmentation in NLP: Best Practices](https://neptune.ai/blog/data-augmentation-nlp)
- [Data Augmentation in NLP: Introduction to Text Augmentation](https://towardsdatascience.com/data-augmentation-in-nlp-2801a34dfc28)
- [Data Augmentation Library for Text](https://towardsdatascience.com/data-augmentation-library-for-text-9661736b13ff)

### Libraries
- [EDA](https://github.com/jasonwei20/eda_nlp): Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks
- [Snorkel Ifor Data Augmentation](https://www.snorkel.org/use-cases/02-spam-data-augmentation-tutorial) (Snorkel can be useful for much more!)

This notebook contains assigments: <font color='red'>Questions are written in red.</font>

<a href="https://colab.research.google.com/github/markif/2021_HS_CAS_NLP_LAB_Notebooks/blob/master/06_a_Data_Augmentation.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%%capture

!pip install 'fhnw-nlp-utils>=0.2.13,<0.3.0'

from fhnw.nlp.utils.processing import parallelize_dataframe
from fhnw.nlp.utils.processing import is_iterable
from fhnw.nlp.utils.storage import download
from fhnw.nlp.utils.storage import save_dataframe
from fhnw.nlp.utils.storage import load_dataframe

import numpy as np
import pandas as pd

In [2]:
from fhnw.nlp.utils.system import system_info
print(system_info())

OS name: posix
Platform name: Linux
Platform release: 5.11.0-40-generic
Python version: 3.6.9
Tensorflow version: 2.5.1
GPU is available


## Data Augmentation

A simple technique to augment a text (i.e. generate a new text from an existing text with (hopefully) the same meaning) is to replace a selected word(s) with a synonym. One possible *snonym provider* are word embeddings (i.e. fasttext.get_nearest_neighbors(word, k=20) as shown above).

- Synonym Replacement (as discussed above)
- Random Insertion (of synonyms - likely to produce grammatical incorrect text) 
- Random Swap (likely to produce grammatical incorrect text - not that useful for BoW models)
- Random Deletion (likely to produce grammatical incorrect text)

In case you work on a character based model it might also be useful to augment text on the character level (e.g. if OCR is part of your pipeline, character level augmentation can help to produce malformed OCR input).

More advanced options include *backtranslation* (e.g. translate text to english and back to german), text summarization, text generation etc.

In [3]:
!pip install nltk

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

A synonym provider building on word embeddings (i.e. fasttext).

In [8]:
%%capture

import nltk 
nltk.download('punkt')

!pip install fasttext

import fasttext
import fasttext.util
from fhnw.nlp.utils.colab import runs_on_colab

if runs_on_colab():
    # colab as problems handling such large files
    model_name = "cc.de.50.bin"
    download("https://drive.google.com/uc?id=1iqw8UPEEVmzQQGmI5FkRJH8B5SkZCgXG", model_name)
else:
    model_name = "cc.de.300.bin"
    fasttext.util.download_model('de', if_exists='ignore')
    
ft = fasttext.load_model(model_name)

<font color='red'>**TASK: Implement `synonym_provider` using fasttext's [`get_nearest_neighbors`](https://fasttext.cc/docs/en/unsupervised-tutorial.html#nearest-neighbor-queries) function (change to the *Python* tab after you opened the link) and provide the functionality as described in the function documentation.**</font>

In [9]:
def synonym_provider(word):
    """Provides a list of synonyms for the given word

    Parameters
    ----------
    word : str
        The word to get the synonym
        
    Returns
    -------
    list
        A list of synonyms for the given word
    """
    
    # TODO: !!! place your code here !!!
    ####################################
    # !!! this needs rework !!!
    synonyms = [word]
    

    ###################
    # TODO: !!! end !!!
    
    return synonyms

In [11]:
synonym_provider("Arzt")

['Hausarzt',
 'Kinderarzt',
 'Ärztin',
 'Augenarzt',
 'Lungenarzt',
 'Frauenarzt',
 'Arztin',
 'Psychiater',
 'Zahnarzt',
 'Mediziner',
 'Röntgenarzt',
 'Tierarzt',
 'arzt',
 'Familienarzt',
 'Chirurg',
 'Vertretungsarzt',
 'Neurologe',
 'HNO-Arzt',
 'Allgemeinmediziner',
 'Krankenhausarzt']

<font color='red'>**Question: Why is it a good idea to use fasttext to find synonyms for German words (i.e. why would word2vec not work that well)? Are there alternatives we could use?**</font>

<font color='green'>**Place your answer here...**</font>

Let's do some augmentation (this **took ~1.5h** on my computer - you might want to examine the code but skip execution).

**Further down there are some examples using [nlpaug](https://github.com/makcedward/nlpaug) which could also be of interest.**

In [ ]:
import random
random.seed(1)

def synonym_replacement(words, unique_words, n, synonym_provider):
    """Replaces words through synonyms

    Parameters
    ----------
    words : list
        The word tokens
    unique_words : set
        The set of unique words
    n : int
        The number of words to replace
    synonym_provider : function
        The function to provide a synonym for a specific word
        
    Returns
    -------
    str
        The new text sequence 
    """
    
    import random
    from random import shuffle
        
    random_words = list(unique_words)
    random.shuffle(random_words)
    random_word = random_words[0]
    synonyms = synonym_provider(random_word)
    #random.shuffle(synonyms)
    sentences = []
    
    for i in range(0, min(n, len(synonyms))):
        synonym = synonyms[i]
        new_words = [synonym if word == random_word else word for word in words]
        sentences.append(' '.join(new_words))

    return sentences

def random_deletion(words, unique_words, n):
    """Randomly deletes words 

    Parameters
    ----------
    words : list
        The word tokens
    unique_words : set
        The set of unique words
    n : int
        The number of words to delete
        
    Returns
    -------
    str
        The new text sequence 
    """
    
    import random
    from random import shuffle
    
    random_words = list(unique_words)
    random.shuffle(random_words)
    sentences = []
    
    for i in range(0, min(n, len(random_words))):
        new_words = words.copy()
        random_word = random_words[i]
        new_words.remove(random_word)
        sentences.append(' '.join(new_words))

    return sentences

def random_swaping(words, unique_words, n):
    """Randomly swaps words

    Parameters
    ----------
    words : list
        The word tokens
    unique_words : set
        The set of unique words
    n : int
        The number of words to swap
        
    Returns
    -------
    str
        The new text sequence 
    """
    
    import random
    from random import shuffle
    
    sentences = []
    if len(unique_words) <= 1:
        return sentences
    
    indices = []
    for i in range(0, len(words)):
        if words[i] in unique_words:
            indices.append(i) 
            
    if len(indices) <= 0:
        return sentences
    
    for i in range(0, n):
        new_words = words.copy()
        idx_1 = random.randint(0, len(new_words)-1)
        word_1 = new_words[idx_1]
        idx_2 = random.randint(0, len(new_words)-1)
        new_words[idx_1] = new_words[idx_2]
        new_words[idx_2] = word_1
        sentences.append(' '.join(new_words))

    return sentences    

def random_insertion(words, unique_words, n, synonym_provider):
    """Randomly inserts n words into the sequence

    Parameters
    ----------
    words : list
        The word tokens
    unique_words : set
        The set of unique words
    n : int
        The number of words to replace
    synonym_provider : function
        The function to provide a synonym for a specific word
        
    Returns
    -------
    str
        The new text sequence 
    """
    
    import random
    from random import shuffle
    
    random_words = list(unique_words)
    random.shuffle(random_words)
    random_word = random_words[0]
    synonyms = synonym_provider(random_word)
    #random.shuffle(synonyms)
    sentences = []
    
    for i in range(0, min(n, len(synonyms))):
        synonym = synonyms[i]
        new_words = words.copy()
        random_idx = random.randint(0, len(new_words)-1)
        new_words.insert(random_idx, synonym)
        sentences.append(' '.join(new_words))

    return sentences

def augment_text(text, stopwords, synonym_provider, synonym_replace=True, random_insert=True, random_swap=True, random_delete=True, num_aug=8):
    """The main augmentation function

    Parameters
    ----------
    text : str
        The text
    stopwords : set
        The set of stopwords
    synonym_provider : function
        The function to provide a synonym for a specific word
    synonym_replace : bool
        Defines if synonym replacement should take place
    random_insert : bool
        Defines if random insert should take place
    random_swap : bool 
        Defines if random swap should take place
    random_delete : bool
        Defines if random deletion should take placd
    num_aug : int
        The number of generated augmented sentences per original sentence
        
    Returns
    -------
    list
        The new generated text sequences 
    """
    
    from fhnw.nlp.utils.processing import is_iterable
    from nltk.tokenize import word_tokenize

    if isinstance(text, str):
        words = word_tokenize(text)
    elif is_iterable(text):
        words = text
    else:
        raise TypeError("Only string or iterable is supported. Received a "+ str(type(text)))
    
    unique_words = set([word for word in words if word not in stopwords])
    if len(unique_words) == 0:
        # stop here
        return []
    
    techniques = 0
    if synonym_replace:
        techniques += 1
    if random_insert:
        techniques += 1
    if random_swap:
        techniques += 1
    if random_delete:
        techniques += 1
        
    augmented_sentences = []
    num_new_per_technique = round(num_aug/techniques)

    if synonym_replace:
        new_sentences = synonym_replacement(words, unique_words, num_new_per_technique, synonym_provider)
        augmented_sentences.extend(new_sentences)

    if random_insert:
        new_sentences = random_insertion(words, unique_words, num_new_per_technique, synonym_provider)
        augmented_sentences.extend(new_sentences)

    if random_swap:
        new_sentences = random_swaping(words, unique_words, num_new_per_technique)
        augmented_sentences.extend(new_sentences)

    if random_delete:
        new_sentences = random_deletion(words, unique_words, num_new_per_technique)
        augmented_sentences.extend(new_sentences)

    return augmented_sentences

def augment_text_df(df, synonym_provider, stopwords=set(), field_read="text", field_label="sentiment"):
    """Augments a column of text by calling augment_text (primarily meant for parallel processing)

    Parameters
    ----------
    df : dataframe
        The dataframe
    synonym_provider : function
        The function to provide a synonym for a specific word
    stopwords : set
        A set of stopword to remove from the tokens
    field_read : str
        The column name to read from (default is text)
    field_label : str
        The column name with the label (default is sentiment)
        
    Returns
    -------
    dataframe
        The dataframe with the normalized text
    """
    
    augmented = []
    
    for index, row in df.iterrows():
        sents = augment_text(row[field_read], stopwords, synonym_provider, num_aug=8, synonym_replace=True, random_insert=True, random_swap=False, random_delete=False)
        rows = [row] * len(sents)
        
        for i in range(0, len(rows)):
            rows[i][field_read] = sents[i]
            
        augmented.extend(rows)
       
    return pd.DataFrame(augmented, columns=df.columns)

In [14]:
stopwords = set(["ich", "bin", "mit", "dieser"])

augment_text("ich bin sehr unzufrieden mit dieser ärztin", stopwords, synonym_provider, num_aug=20)

['ich bin äußerst unzufrieden mit dieser ärztin',
 'ich bin extrem unzufrieden mit dieser ärztin',
 'ich bin überaus unzufrieden mit dieser ärztin',
 'ich bin außerordentlich unzufrieden mit dieser ärztin',
 'ich bin ziemlich unzufrieden mit dieser ärztin',
 'ich bin sehr zufrieden unzufrieden mit dieser ärztin',
 'ich bin sehr unzufrieden mit dieser Unzufrieden ärztin',
 'ich bin sehr frustriert unzufrieden mit dieser ärztin',
 'ich bin sehr enttäuscht unzufrieden mit dieser ärztin',
 'ich bin sehr unzufrieden mit unzufriedenstellend dieser ärztin',
 'ich bin sehr ärztin mit dieser unzufrieden',
 'bin ich sehr unzufrieden mit dieser ärztin',
 'unzufrieden bin sehr ich mit dieser ärztin',
 'ich bin sehr ärztin mit dieser unzufrieden',
 'ich bin sehr mit unzufrieden dieser ärztin',
 'ich bin sehr mit dieser ärztin',
 'ich bin unzufrieden mit dieser ärztin',
 'ich bin sehr unzufrieden mit dieser']

In [16]:
%%time
download("https://drive.google.com/uc?id=19AFeVnOfX8WXU4_3rM7OFoNTWWog_sb_", "data/german_doctor_reviews_tokenized.parq")
data = load_dataframe("data/german_doctor_reviews_tokenized.parq")
data.shape

CPU times: user 8.15 s, sys: 1.46 s, total: 9.6 s
Wall time: 5.6 s


(350087, 10)

In [17]:
data.head(3)

,text_original,rating,text,label,sentiment,token_clean,text_clean,token_lemma,token_stem,token_clean_stopwords
0,Ich bin franzose und bin seit ein paar Wochen ...,2.0,Ich bin franzose und bin seit ein paar Wochen ...,positive,1,"[ich, bin, franzose, und, bin, seit, ein, paar...",ich bin franzose und bin seit ein paar wochen ...,"[franzose, seit, paar, wochen, muenchen, zahn,...","[franzos, seit, paar, woch, muench, ., zahn, s...","[franzose, seit, paar, wochen, muenchen, ., za..."
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1,"[dieser, arzt, ist, das, unmöglichste, was, mi...",dieser arzt ist das unmöglichste was mir in me...,"[arzt, unmöglichste, leben, je, begegnen, unfr...","[arzt, unmog, leb, je, begegnet, unfreund, ,, ...","[arzt, unmöglichste, leben, je, begegnet, unfr..."
2,Hatte akute Beschwerden am Rücken. Herr Magura...,1.0,Hatte akute Beschwerden am Rücken. Herr Magura...,positive,1,"[hatte, akute, beschwerden, am, rücken, ., her...",hatte akute beschwerden am rücken . herr magur...,"[akut, beschwerden, rücken, magura, erste, arz...","[akut, beschwerd, ruck, ., magura, erst, arzt,...","[akute, beschwerden, rücken, ., magura, erste,..."


In [18]:
data = data.drop(["token_clean", "token_lemma", "token_stem", "token_clean_stopwords", "text_clean"], axis=1)

In [19]:
data.head(3)

,text_original,rating,text,label,sentiment
0,Ich bin franzose und bin seit ein paar Wochen ...,2.0,Ich bin franzose und bin seit ein paar Wochen ...,positive,1
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
2,Hatte akute Beschwerden am Rücken. Herr Magura...,1.0,Hatte akute Beschwerden am Rücken. Herr Magura...,positive,1


In [20]:
# only keep negative text (the class with fewer samples)
data_augm = data[data["label"] == "negative"]

data_augm.head(3)

,text_original,rating,text,label,sentiment
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
13,1. Termin:<br />\n1 Stunde Wartezimmer + 2 min...,6.0,. Termin Stunde Wartezimmer minütige Behandlu...,negative,-1
19,"Eine sehr unfreundliche Ärztin, so etwas habe ...",6.0,"Eine sehr unfreundliche Ärztin, so etwas habe ...",negative,-1


In [18]:
import nltk

from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

stopwords = set(stopwords.words("german"))

stopwords.add("frau")
stopwords.add("dr")
stopwords.add("herr")
stopwords.add("herrn")
stopwords.add("fr")
stopwords.add("werden")

stopwords.remove("nicht")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
%%time

data_test = data_augm.head(2)
augment_text_df(data_test, synonym_provider=synonym_provider, stopwords=stopwords)

CPU times: user 2.98 s, sys: 5.18 ms, total: 2.99 s
Wall time: 2.98 s


,text_original,rating,text,label,sentiment
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
13,1. Termin:<br />\n1 Stunde Wartezimmer + 2 min...,6.0,. Termin Stunde Wartezimmer minütige Behandlun...,negative,-1
13,1. Termin:<br />\n1 Stunde Wartezimmer + 2 min...,6.0,. Termin Stunde Wartezimmer minütige Behandlun...,negative,-1


In [ ]:
%%time

# this will take ~1.5 h
data_augmented = parallelize_dataframe(data_augm, augment_text_df, synonym_provider=synonym_provider, stopwords=stopwords, processing_mode="df")

In [5]:
#data_augmented = load_dataframe("data/german_doctor_reviews_augmented.parq")

In [7]:
data_augmented[data_augmented["text"].str.contains("<", na=False)]

,text_original,rating,sentiment,text


In [ ]:
%%time

from fhnw.nlp.utils.text import clean_text

# make sure we did not introduce maleformated stuff
data_augmented = data_augmented.rename(columns={"text": "text_tmp"})
data_augmented = parallelize_dataframe(data_augmented, clean_text, field_read="text_original", field_write="text", keep_punctuation=True)
data_augmented = data_augmented.drop(columns=["text_tmp"], errors="ignore")

In [ ]:
data_augmented.head(3)

In [19]:
save_dataframe(data_augmented, "data/german_doctor_reviews_augmented.parq")

## nlpaug

nlpaug is an augmentation library for text (word and character level) and speech and can do much more...

see:
- https://github.com/makcedward/nlpaug
- https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb
- https://neptune.ai/blog/data-augmentation-nlp -> NLPAug Library

nlpaug can much more...

In [23]:
!pip install nlpaug

     |████████████████████████████████| 405 kB 3.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [24]:
#%load_ext autoreload
#%autoreload 2
#import importlib

In [25]:
import os
os.environ["MODEL_DIR"] = '../model'

In [26]:
import nlpaug.augmenter.word as naw

### Contextual Word Embeddings

Hmmm...

Well, it does not know the context (doctor reviews) of our text...

In [27]:
# see https://huggingface.co/models for possible models

#aug = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-uncased', stopwords=stopwords, aug_min=2, device="cuda")
#aug = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-uncased', stopwords=stopwords, aug_min=1)
#aug = naw.ContextualWordEmbsAug(model_path='distilbert-base-multilingual-cased', aug_min=2)
aug = naw.ContextualWordEmbsAug(model_path='distilbert-base-german-cased', stopwords=stopwords, aug_min=2)

Downloading:   0%|          | 0.00/464 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/270M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/240k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479k [00:00<?, ?B/s]

In [28]:
text = "Ich bin sehr unzufrieden mit diesem Arzt."
text_aug = aug.augment(text)
print("Original:", text)
print("Augmented:", text_aug)

Original: Ich bin sehr unzufrieden mit diesem Arzt.
Augmented: Mir bin sehr unzufrieden mit diesem Posten.


### Backtranslation

:-)

In [29]:
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-de-en',
    to_model_name='facebook/wmt19-en-de',
    #device='cuda'
)

Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/315k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/849k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/849k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315k [00:00<?, ?B/s]

In [30]:
text = "Ich bin sehr unzufrieden mit diesem Arzt."
text_aug = back_translation_aug.augment(text)
print("Original:", text)
print("Augmented:", text_aug)

Original: Ich bin sehr unzufrieden mit diesem Arzt.
Augmented: Ich bin sehr unglücklich mit diesem Arzt.


### Text Generation

Add sentences using text generation. Here we would need a model that was trained ou our corpus...

In [38]:
import nlpaug.augmenter.sentence as nas

aug = nas.ContextualWordEmbsForSentenceAug(model_path='dbmdz/german-gpt2')

In [40]:
text = "Ich bin sehr unzufrieden mit diesem Arzt."
text_aug = aug.augment(text)
print("Original:", text)
print("Augmented:", text_aug)

Using pad_token, but it is not set yet.


Original: Ich bin sehr unzufrieden mit diesem Arzt.
Augmented: Ich bin sehr unzufrieden mit diesem Arzt.
Oh, nein, er ist sehr krank.
- Aber, Dr. Jackson.
- Nein, er hat einen wichtigen Eingriff hinter sich.
Er hat immer noch keine Ahnung.
Oh, mein Gott, nein, es ist doch ein ganz schöner Eingriff.
- Wir werden noch mehr Untersuchungen machen.
- Oh, Dr. Sloan, kommen Sie, wir machen eine Liste mit Ärzten.
Das sind nur Ärzte, keine Krankenschwestern.
Sie kommen auf diese Liste, weil sie ihre Arbeit nicht erledigen können.
Oh, Dr. Jackson, wir werden die Liste weiterreichen.
Okay.
Aber wir haben gerade ein neues Experiment gemacht.
Und wir müssen dieses Teil wegräumen, bis sie wissen, was passiert ist.
Und das ist wichtig, um die Ergebnisse des Experiments zu überprüfen.
Bitte, das ist lächerlich.
Sie kommen auf die Liste zurück.
Ich kann es fühlen, wie Sie es fühlen.
- Was?
- Sie sind Ärztin.
Ich bin Ärztin, nicht Arzt.
Das kann ich nicht machen.
Wenn Sie mir helfen können, helfen Sie m

### Summarization


In [33]:
aug = nas.AbstSummAug(model_path='t5-base')

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [35]:
text = "Ich bin sehr unzufrieden mit diesem Arzt."
text_aug = aug.augment(text)
print("Original:", text)
print("Augmented:", text_aug)

Your max_length is set to 50, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Original: Ich bin sehr unzufrieden mit diesem Arzt.
Augmented: Ich bin sehr unzufrieden mit diesem Arzt. Ich habe es nicht geschafft, meinen Arzt zu ersetzen.


In [36]:
text = data.iloc[20758]["text"]
text_aug = aug.augment(text)
print("Original:", text)
print("Augmented:", text_aug)

Original: Ich empfinde Dr. Niedenthal als freundlichen und kompetenten Arzt, bei dem ich schon seit einigen Jahren bin. Bisher hat er mich auch noch nicht zu unnötigen und teuren Behandlungen überredet. Auch Preis Leistung für professionelle Zahnreinigung finde ich in Ordnung. Negativ Anmerken muss ich die teilweise langen Wartezeiten vor Ort. Min. im Wartezimmer ist in Ordnung, aber Min. zusätzliches warten im Behandlungsraum ist es nicht! Beim letzten Mal ist Dr. Niedenthal auch während der Behandlung für rund Min. verschwunden. Das ist nicht in Ordnung! Vor einer Behandlung wäre es weiterhin angebracht, den Patienten kurz über die Zusatzkosten zu informieren. War bei mir nicht viel, jedoch war ich trotzdem über die Rechnung in der Post überrascht.
Augmented: Dr. Niedenthal ist ein freundlicher und kompetenter Arzt, der mich auch noch nicht zu unnötigen und teuren Behandlungen überredet hat.


In [37]:
text = data.iloc[313390]["text"]
text_aug = aug.augment(text)
print("Original:", text)
print("Augmented:", text_aug)

Original: Der Arzt ist sehr kompetent und die Behandlung ist sehr gut , seine Art iat zwar etwas eigen aber das nehme ich gern in Kauf Die Schwestern sind größtenteils nett bis auf eune Dame die ich nur schlecht gelaunt und intrigant erlebt habe.
Augmented: Der Arzt ist sehr kompetent und die Behandlung ist sehr gut, seine Art iat zwar etwas eigen aber das nehme ich gern in Kauf Die Schwestern sind größtenteils nett bis auf 


### Text Augmentation with nlpaug

In [41]:
def nlpaug_augment_df(df, augmenter, field_read="text", field_label="sentiment"):
    augmented = []
    
    for index, row in df.iterrows():
        sent = augmenter(row[field_read])
        copy_row = row.copy()
        copy_row[field_read] = sent
            
        augmented.append(copy_row)
        
        if len(augmented) % 2 is 0:
            print("Store", len(augmented), "of", df.shape[0])
            save_dataframe(pd.DataFrame(augmented, columns=df.columns), "data/german_doctor_reviews_augmented_translated_part.parq")
       
    return pd.DataFrame(augmented, columns=df.columns)

In [42]:
augmenter = lambda text: back_translation_aug.augment(text)

In [43]:
text = data.iloc[0]["text"]
translation = augmenter(text)

print(text)
print(translation)

Ich bin franzose und bin seit ein paar Wochen in muenchen. Ich hatte Zahn Schmerzen und mein Kollegue hat mir Dr mainka empfohlen. Ich habe schnell ein Termin bekommen, das Team war nett und meine schmerzen sind weg!! Ich bin als Angst Patient sehr zurieden!!
Ich bin Franzose und seit ein paar Wochen in München. Ich hatte Zahnschmerzen und mein Kollege empfahl mich Dr. mainka. Ich bekam schnell einen Termin, das Team war nett und meine Schmerzen sind weg!! Ich bin eine sehr ängstliche Patientin!!


In [44]:
text = data.iloc[1]["text"]
translation = augmenter(text)

print(text)
print(translation)

Dieser Arzt ist das unmöglichste was mir in meinem Leben je begegnet ist er ist unfreundlich ,sehr herablassend und medizinisch unkompetent Nach seiner Diagnose bin ich zu einem anderen Hautarzt gegangen der mich ordentlich behandelt hat und mir auch half Meine Beschweerden hatten einen völlig anderen Grund. Nach seiner Behandlung und Diagnose ,waren seine letzten Worte .....und tschüss Alles inerhalb von ca Minuten.
Dieser Arzt ist das Unmöglichste, was mir je in meinem Leben begegnet ist, er ist unfreundlich, sehr herablassend und medizinisch inkompetent. Nach seiner Diagnose ging ich zu einem anderen Hautarzt, der mich ordnungsgemäß behandelte und mir auch half Meine Wunden hatten einen ganz anderen Grund. Nach seiner Behandlung und Diagnose waren seine letzten Worte..... und verabschiedeten sich innerhalb einer Minute.


In [45]:
text = data.iloc[2]["text"]
translation = augmenter(text)

print(text)
print(translation)

Hatte akute Beschwerden am Rücken. Herr Magura war der erste Arzt der sich wirklich Zeit für einen Therapieplan genommen hat um nachhaltig meine Schmerzen zu beseitigen
Herr Magura war der erste Arzt, der sich wirklich die Zeit für einen Therapieplan nahm, um meine Schmerzen dauerhaft zu beseitigen.


In [46]:
data_translated  = data[data["sentiment"] == -1]

data_translated.head(3)

,text_original,rating,text,label,sentiment
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1
13,1. Termin:<br />\n1 Stunde Wartezimmer + 2 min...,6.0,. Termin Stunde Wartezimmer minütige Behandlu...,negative,-1
19,"Eine sehr unfreundliche Ärztin, so etwas habe ...",6.0,"Eine sehr unfreundliche Ärztin, so etwas habe ...",negative,-1


In [17]:
%%time
# parallelization does not seem to provide a boost
data_translated = nlpaug_augment_df(data_translated, augmenter=augmenter)

# could not use GPU
# and took incredible 4d 2h 2min 42s

Store 2 of 33023
Store 4 of 33023
Store 6 of 33023
Store 8 of 33023
Store 10 of 33023
Store 12 of 33023
Store 14 of 33023
Store 16 of 33023
Store 18 of 33023
Store 20 of 33023
Store 22 of 33023
Store 24 of 33023
Store 26 of 33023
Store 28 of 33023
Store 30 of 33023
Store 32 of 33023
Store 34 of 33023
Store 36 of 33023
Store 38 of 33023
Store 40 of 33023
Store 42 of 33023
Store 44 of 33023
Store 46 of 33023
Store 48 of 33023
Store 50 of 33023
Store 52 of 33023
Store 54 of 33023
Store 56 of 33023
Store 58 of 33023
Store 60 of 33023
Store 62 of 33023
Store 64 of 33023
Store 66 of 33023
Store 68 of 33023
Store 70 of 33023
Store 72 of 33023
Store 74 of 33023
Store 76 of 33023
Store 78 of 33023
Store 80 of 33023
Store 82 of 33023
Store 84 of 33023
Store 86 of 33023
Store 88 of 33023
Store 90 of 33023
Store 92 of 33023
Store 94 of 33023
Store 96 of 33023
Store 98 of 33023
Store 100 of 33023
Store 102 of 33023
Store 104 of 33023
Store 106 of 33023
Store 108 of 33023
Store 110 of 33023
Store 11

Store 870 of 33023
Store 872 of 33023
Store 874 of 33023
Store 876 of 33023
Store 878 of 33023
Store 880 of 33023
Store 882 of 33023
Store 884 of 33023
Store 886 of 33023
Store 888 of 33023
Store 890 of 33023
Store 892 of 33023
Store 894 of 33023
Store 896 of 33023
Store 898 of 33023
Store 900 of 33023
Store 902 of 33023
Store 904 of 33023
Store 906 of 33023
Store 908 of 33023
Store 910 of 33023
Store 912 of 33023
Store 914 of 33023
Store 916 of 33023
Store 918 of 33023
Store 920 of 33023
Store 922 of 33023
Store 924 of 33023
Store 926 of 33023
Store 928 of 33023
Store 930 of 33023
Store 932 of 33023
Store 934 of 33023
Store 936 of 33023
Store 938 of 33023
Store 940 of 33023
Store 942 of 33023
Store 944 of 33023
Store 946 of 33023
Store 948 of 33023
Store 950 of 33023
Store 952 of 33023
Store 954 of 33023
Store 956 of 33023
Store 958 of 33023
Store 960 of 33023
Store 962 of 33023
Store 964 of 33023
Store 966 of 33023
Store 968 of 33023
Store 970 of 33023
Store 972 of 33023
Store 974 of

Store 1696 of 33023
Store 1698 of 33023
Store 1700 of 33023
Store 1702 of 33023
Store 1704 of 33023
Store 1706 of 33023
Store 1708 of 33023
Store 1710 of 33023
Store 1712 of 33023
Store 1714 of 33023
Store 1716 of 33023
Store 1718 of 33023
Store 1720 of 33023
Store 1722 of 33023
Store 1724 of 33023
Store 1726 of 33023
Store 1728 of 33023
Store 1730 of 33023
Store 1732 of 33023
Store 1734 of 33023
Store 1736 of 33023
Store 1738 of 33023
Store 1740 of 33023
Store 1742 of 33023
Store 1744 of 33023
Store 1746 of 33023
Store 1748 of 33023
Store 1750 of 33023
Store 1752 of 33023
Store 1754 of 33023
Store 1756 of 33023
Store 1758 of 33023
Store 1760 of 33023
Store 1762 of 33023
Store 1764 of 33023
Store 1766 of 33023
Store 1768 of 33023
Store 1770 of 33023
Store 1772 of 33023
Store 1774 of 33023
Store 1776 of 33023
Store 1778 of 33023
Store 1780 of 33023
Store 1782 of 33023
Store 1784 of 33023
Store 1786 of 33023
Store 1788 of 33023
Store 1790 of 33023
Store 1792 of 33023
Store 1794 of 33023


Store 2516 of 33023
Store 2518 of 33023
Store 2520 of 33023
Store 2522 of 33023
Store 2524 of 33023
Store 2526 of 33023
Store 2528 of 33023
Store 2530 of 33023
Store 2532 of 33023
Store 2534 of 33023
Store 2536 of 33023
Store 2538 of 33023
Store 2540 of 33023
Store 2542 of 33023
Store 2544 of 33023
Store 2546 of 33023
Store 2548 of 33023
Store 2550 of 33023
Store 2552 of 33023
Store 2554 of 33023
Store 2556 of 33023
Store 2558 of 33023
Store 2560 of 33023
Store 2562 of 33023
Store 2564 of 33023
Store 2566 of 33023
Store 2568 of 33023
Store 2570 of 33023
Store 2572 of 33023
Store 2574 of 33023
Store 2576 of 33023
Store 2578 of 33023
Store 2580 of 33023
Store 2582 of 33023
Store 2584 of 33023
Store 2586 of 33023
Store 2588 of 33023
Store 2590 of 33023
Store 2592 of 33023
Store 2594 of 33023
Store 2596 of 33023
Store 2598 of 33023
Store 2600 of 33023
Store 2602 of 33023
Store 2604 of 33023
Store 2606 of 33023
Store 2608 of 33023
Store 2610 of 33023
Store 2612 of 33023
Store 2614 of 33023


Store 3336 of 33023
Store 3338 of 33023
Store 3340 of 33023
Store 3342 of 33023
Store 3344 of 33023
Store 3346 of 33023
Store 3348 of 33023
Store 3350 of 33023
Store 3352 of 33023
Store 3354 of 33023
Store 3356 of 33023
Store 3358 of 33023
Store 3360 of 33023
Store 3362 of 33023
Store 3364 of 33023
Store 3366 of 33023
Store 3368 of 33023
Store 3370 of 33023
Store 3372 of 33023
Store 3374 of 33023
Store 3376 of 33023
Store 3378 of 33023
Store 3380 of 33023
Store 3382 of 33023
Store 3384 of 33023
Store 3386 of 33023
Store 3388 of 33023
Store 3390 of 33023
Store 3392 of 33023
Store 3394 of 33023
Store 3396 of 33023
Store 3398 of 33023
Store 3400 of 33023
Store 3402 of 33023
Store 3404 of 33023
Store 3406 of 33023
Store 3408 of 33023
Store 3410 of 33023
Store 3412 of 33023
Store 3414 of 33023
Store 3416 of 33023
Store 3418 of 33023
Store 3420 of 33023
Store 3422 of 33023
Store 3424 of 33023
Store 3426 of 33023
Store 3428 of 33023
Store 3430 of 33023
Store 3432 of 33023
Store 3434 of 33023


...

Store 29450 of 33023
Store 29452 of 33023
Store 29454 of 33023
Store 29456 of 33023
Store 29458 of 33023
Store 29460 of 33023
Store 29462 of 33023
Store 29464 of 33023
Store 29466 of 33023
Store 29468 of 33023
Store 29470 of 33023
Store 29472 of 33023
Store 29474 of 33023
Store 29476 of 33023
Store 29478 of 33023
Store 29480 of 33023
Store 29482 of 33023
Store 29484 of 33023
Store 29486 of 33023
Store 29488 of 33023
Store 29490 of 33023
Store 29492 of 33023
Store 29494 of 33023
Store 29496 of 33023
Store 29498 of 33023
Store 29500 of 33023
Store 29502 of 33023
Store 29504 of 33023
Store 29506 of 33023
Store 29508 of 33023
Store 29510 of 33023
Store 29512 of 33023
Store 29514 of 33023
Store 29516 of 33023
Store 29518 of 33023
Store 29520 of 33023
Store 29522 of 33023
Store 29524 of 33023
Store 29526 of 33023
Store 29528 of 33023
Store 29530 of 33023
Store 29532 of 33023
Store 29534 of 33023
Store 29536 of 33023
Store 29538 of 33023
Store 29540 of 33023
Store 29542 of 33023
Store 29544 o

Store 30232 of 33023
Store 30234 of 33023
Store 30236 of 33023
Store 30238 of 33023
Store 30240 of 33023
Store 30242 of 33023
Store 30244 of 33023
Store 30246 of 33023
Store 30248 of 33023
Store 30250 of 33023
Store 30252 of 33023
Store 30254 of 33023
Store 30256 of 33023
Store 30258 of 33023
Store 30260 of 33023
Store 30262 of 33023
Store 30264 of 33023
Store 30266 of 33023
Store 30268 of 33023
Store 30270 of 33023
Store 30272 of 33023
Store 30274 of 33023
Store 30276 of 33023
Store 30278 of 33023
Store 30280 of 33023
Store 30282 of 33023
Store 30284 of 33023
Store 30286 of 33023
Store 30288 of 33023
Store 30290 of 33023
Store 30292 of 33023
Store 30294 of 33023
Store 30296 of 33023
Store 30298 of 33023
Store 30300 of 33023
Store 30302 of 33023
Store 30304 of 33023
Store 30306 of 33023
Store 30308 of 33023
Store 30310 of 33023
Store 30312 of 33023
Store 30314 of 33023
Store 30316 of 33023
Store 30318 of 33023
Store 30320 of 33023
Store 30322 of 33023
Store 30324 of 33023
Store 30326 o

Store 31014 of 33023
Store 31016 of 33023
Store 31018 of 33023
Store 31020 of 33023
Store 31022 of 33023
Store 31024 of 33023
Store 31026 of 33023
Store 31028 of 33023
Store 31030 of 33023
Store 31032 of 33023
Store 31034 of 33023
Store 31036 of 33023
Store 31038 of 33023
Store 31040 of 33023
Store 31042 of 33023
Store 31044 of 33023
Store 31046 of 33023
Store 31048 of 33023
Store 31050 of 33023
Store 31052 of 33023
Store 31054 of 33023
Store 31056 of 33023
Store 31058 of 33023
Store 31060 of 33023
Store 31062 of 33023
Store 31064 of 33023
Store 31066 of 33023
Store 31068 of 33023
Store 31070 of 33023
Store 31072 of 33023
Store 31074 of 33023
Store 31076 of 33023
Store 31078 of 33023
Store 31080 of 33023
Store 31082 of 33023
Store 31084 of 33023
Store 31086 of 33023
Store 31088 of 33023
Store 31090 of 33023
Store 31092 of 33023
Store 31094 of 33023
Store 31096 of 33023
Store 31098 of 33023
Store 31100 of 33023
Store 31102 of 33023
Store 31104 of 33023
Store 31106 of 33023
Store 31108 o

Store 31796 of 33023
Store 31798 of 33023
Store 31800 of 33023
Store 31802 of 33023
Store 31804 of 33023
Store 31806 of 33023
Store 31808 of 33023
Store 31810 of 33023
Store 31812 of 33023
Store 31814 of 33023
Store 31816 of 33023
Store 31818 of 33023
Store 31820 of 33023
Store 31822 of 33023
Store 31824 of 33023
Store 31826 of 33023
Store 31828 of 33023
Store 31830 of 33023
Store 31832 of 33023
Store 31834 of 33023
Store 31836 of 33023
Store 31838 of 33023
Store 31840 of 33023
Store 31842 of 33023
Store 31844 of 33023
Store 31846 of 33023
Store 31848 of 33023
Store 31850 of 33023
Store 31852 of 33023
Store 31854 of 33023
Store 31856 of 33023
Store 31858 of 33023
Store 31860 of 33023
Store 31862 of 33023
Store 31864 of 33023
Store 31866 of 33023
Store 31868 of 33023
Store 31870 of 33023
Store 31872 of 33023
Store 31874 of 33023
Store 31876 of 33023
Store 31878 of 33023
Store 31880 of 33023
Store 31882 of 33023
Store 31884 of 33023
Store 31886 of 33023
Store 31888 of 33023
Store 31890 o

Store 32578 of 33023
Store 32580 of 33023
Store 32582 of 33023
Store 32584 of 33023
Store 32586 of 33023
Store 32588 of 33023
Store 32590 of 33023
Store 32592 of 33023
Store 32594 of 33023
Store 32596 of 33023
Store 32598 of 33023
Store 32600 of 33023
Store 32602 of 33023
Store 32604 of 33023
Store 32606 of 33023
Store 32608 of 33023
Store 32610 of 33023
Store 32612 of 33023
Store 32614 of 33023
Store 32616 of 33023
Store 32618 of 33023
Store 32620 of 33023
Store 32622 of 33023
Store 32624 of 33023
Store 32626 of 33023
Store 32628 of 33023
Store 32630 of 33023
Store 32632 of 33023
Store 32634 of 33023
Store 32636 of 33023
Store 32638 of 33023
Store 32640 of 33023
Store 32642 of 33023
Store 32644 of 33023
Store 32646 of 33023
Store 32648 of 33023
Store 32650 of 33023
Store 32652 of 33023
Store 32654 of 33023
Store 32656 of 33023
Store 32658 of 33023
Store 32660 of 33023
Store 32662 of 33023
Store 32664 of 33023
Store 32666 of 33023
Store 32668 of 33023
Store 32670 of 33023
Store 32672 o

In [47]:
data_translated.head(3)

,text_original,rating,text,sentiment
1,dieser arzt ist das unmöglichste was mir in me...,6.0,"dieser arzt ist das Unmögliche, was ich je in ...",-1
13,. termin stunde wartezimmer minütige behandlun...,6.0,. terminstunde wartesaal Minuten der behandlun...,-1
19,"eine sehr unfreundliche ärztin , so etwas habe...",6.0,"eine sehr unfreundliche ärztin, so etwas habe ...",-1


In [42]:
save_dataframe(data_translated, "data/german_doctor_reviews_augmented_translated.parq")

Normalize the augmented data

In [5]:
from fhnw.nlp.utils.normalize import tokenize
from fhnw.nlp.utils.normalize import tokenize_stem
from fhnw.nlp.utils.normalize import tokenize_lemma
from fhnw.nlp.utils.normalize import normalize
from fhnw.nlp.utils.normalize import normalize_df
from fhnw.nlp.utils.text import join_tokens_df

In [6]:
!pip install 'spacy>=3.0.5'
!pip install nltk

import nltk
from nltk.stem.snowball import SnowballStemmer

import spacy
!python3 -m spacy download de_core_news_lg

nlp = spacy.load("de_core_news_lg")

stemmer = SnowballStemmer("german")
empty_stopwords = set()

     |████████████████████████████████| 82 kB 260 kB/s eta 0:00:01
  Attempting uninstall: click
    Found existing installation: click 8.0.1
    Uninstalling click-8.0.1:
      Successfully uninstalled click-8.0.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
2021-09-18 11:09:45.215334: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 571.2 MB 52 kB/s  eta 0:00:01     |█████████████████████████████▉  | 532.7 MB 5.3 MB/s eta 0:00:08


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')


In [7]:
data_augmented = load_dataframe("data/german_doctor_reviews_augmented.parq")
data_translated = load_dataframe("data/german_doctor_reviews_augmented_translated.parq")

In [8]:
data_augmented

,text_original,rating,sentiment,text
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,-1,Dieser Arzt ist das unmöglichste was mir in me...
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,-1,Dieser Arzt ist das unmöglichste was mir in me...
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,-1,Dieser Arzt ist das unmöglichste was mir in me...
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,-1,Dieser Arzt ist das unmöglichste was mir in me...
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,-1,Dieser Arzt ist das unmöglichste was mir in me...
...,...,...,...,...
357887,Unglaublich unfreundliche Praxis.<br />\nDie S...,5.0,-1,Unglaublich unfreundliche Praxis. Die Sprechst...
357887,Unglaublich unfreundliche Praxis.<br />\nDie S...,5.0,-1,Unglaublich unfreundliche Praxis. Die Sprechst...
357887,Unglaublich unfreundliche Praxis.<br />\nDie S...,5.0,-1,Unglaublich unfreundliche Praxis. Die Sprechst...
357887,Unglaublich unfreundliche Praxis.<br />\nDie S...,5.0,-1,Unglaublich unfreundliche Praxis. Die Sprechst...


In [9]:
data_translated

,text_original,rating,text,sentiment
1,dieser arzt ist das unmöglichste was mir in me...,6.0,"dieser arzt ist das Unmögliche, was ich je in ...",-1
13,. termin stunde wartezimmer minütige behandlun...,6.0,. terminstunde wartesaal Minuten der behandlun...,-1
19,"eine sehr unfreundliche ärztin , so etwas habe...",6.0,"eine sehr unfreundliche ärztin, so etwas habe ...",-1
22,es gibt genau zwei ! arzthelferinnen die nett ...,6.0,"es sind genau zwei! arzthelferinnen, die freun...",-1
31,eine unfreundliche ärztin . die nur interesse ...,6.0,ein unfreundlicher Arzt.. der sich nur für das...,-1
...,...,...,...,...
357842,das personal ist sehr jung und denkt nicht dar...,6.0,Das Personal ist sehr jung und denkt nicht dar...,-1
357870,ich besuchte herrn doktor gent zum ersten mal ...,5.0,"ich war das erste mal bei arzt gent, wegen ein...",-1
357874,"dieser arzt ist launisch , unfreundlich und nu...",5.0,"dieser arzt ist launisch, unfreundlich und nur...",-1
357875,der arzt fertigt einen ab . alles fliesbandarb...,6.0,der arzt nimmt dich ab. alles förderband. für ...,-1


In [10]:
data_aug = pd.concat([data_augmented, data_translated])

In [11]:
if "label" not in data_aug.columns:
    data_aug["label"] = "negative"

In [12]:
%%time
data_aug = parallelize_dataframe(data_aug, normalize_df, field_read="text", field_write="token_clean", stopwords=empty_stopwords, stemmer=None, lemmanizer=None, lemma_with_ner=False)

CPU times: user 12.8 s, sys: 2.26 s, total: 15 s
Wall time: 56.4 s


In [13]:
%%time
data_aug = parallelize_dataframe(data_aug, join_tokens_df, field_read="token_clean", field_write="text_clean", stopwords=empty_stopwords)

CPU times: user 1min 31s, sys: 3.88 s, total: 1min 35s
Wall time: 1min 36s


In [19]:
%%time
data_aug = parallelize_dataframe(data_aug, normalize_df, field_read="token_clean", field_write="token_lemma", stopwords=stopwords, stemmer=None, lemmanizer=nlp, lemma_with_ner=False)

CPU times: user 3min 57s, sys: 26 s, total: 4min 23s
Wall time: 6min 42s


In [20]:
%%time
data_aug = parallelize_dataframe(data_aug, normalize_df, field_read="token_clean", field_write="token_stem", stopwords=stopwords, stemmer=stemmer, lemmanizer=None, lemma_with_ner=False)

CPU times: user 1min 37s, sys: 4.79 s, total: 1min 42s
Wall time: 1min 53s


In [21]:
%%time
data_aug = parallelize_dataframe(data_aug, normalize_df, field_read="token_clean", field_write="token_clean_stopwords", stopwords=stopwords, stemmer=None, lemmanizer=None, lemma_with_ner=False)

CPU times: user 1min 40s, sys: 5.14 s, total: 1min 45s
Wall time: 1min 51s


In [22]:
data_aug = data_aug[data_aug["token_lemma"].map(len) > 1 ]

In [23]:
data_aug.head(3)

,text_original,rating,sentiment,text,label,token_clean,text_clean,token_lemma,token_stem,token_clean_stopwords
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,-1,Dieser Arzt ist das unmöglichste was mir in me...,negative,"[dieser, arzt, ist, das, unmöglichste, was, mi...",dieser arzt ist das unmöglichste was mir in me...,"[arzt, unmöglichste, leben, je, begegnen, unfr...","[arzt, unmog, leb, je, begegnet, unfreund, ,, ...","[arzt, unmöglichste, leben, je, begegnet, unfr..."
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,-1,Dieser Arzt ist das unmöglichste was mir in me...,negative,"[dieser, arzt, ist, das, unmöglichste, was, mi...",dieser arzt ist das unmöglichste was mir in me...,"[arzt, unmöglichste, leben, je, begegnen, unfr...","[arzt, unmog, leb, je, begegnet, unfreund, ,, ...","[arzt, unmöglichste, leben, je, begegnet, unfr..."
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,-1,Dieser Arzt ist das unmöglichste was mir in me...,negative,"[dieser, arzt, ist, das, unmöglichste, was, mi...",dieser arzt ist das unmöglichste was mir in me...,"[arzt, unmöglichste, leben, je, begegnen, unfr...","[arzt, unmog, leb, je, begegnet, unfreund, ,, ...","[arzt, unmöglichste, leben, je, begegnet, unfr..."


In [24]:
%%time
save_dataframe(data_aug, "data/german_doctor_reviews_augmented_tokenized.parq")

CPU times: user 25.6 s, sys: 1.04 s, total: 26.7 s
Wall time: 26.6 s


Let's see if batch processing would speed up translation.

Also see https://www.machinecurve.com/index.php/2021/02/16/easy-machine-translation-with-machine-learning-and-huggingface-transformers/

In [9]:
from transformers import pipeline

In [10]:
# Init translator
translator = pipeline("translation_en_to_de")

In [11]:
# Translate text
text = "Hello my friends! How are you doing today?"
translation = translator(text)

/usr/local/lib/python3.6/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [12]:
# Print translation
print(translation)

[{'translation_text': 'Hallo liebe Freunde, wie geht es Ihnen heute?'}]


In [15]:
%%time

translation = translator(data_augm.head(20)["text_clean"].tolist(), max_length=700)

CPU times: user 5min 37s, sys: 2min 17s, total: 7min 54s
Wall time: 1min 28s
